In [2]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as bs

In [2]:
url = 'https://www.expat-dakar.com/voitures'
res = get(url)
soup = bs(res.text, 'html.parser')
containers = soup.find_all('div', class_ = 'listings-cards__list-item')

In [3]:
data = []

for container in containers:
    try:
        # Récupération de l'URL
        url_container = container.find('a', class_='listing-card__inner')['href']
        res_c = get(url_container)
        soup_c = bs(res_c.text, 'html.parser')

        # Récupération des éléments dd
        dd_elements = soup_c.find_all('dd', class_="listing-item__properties__description")
        if len(dd_elements) < 5:
            print("Éléments manquants dans le conteneur")
            continue

        marque = dd_elements[0].text.strip()
        modele = dd_elements[1].text.strip()
        etat = dd_elements[2].text.strip()
        transmission = dd_elements[3].text.strip()
        annee = dd_elements[4].text.strip()

        #
        price = soup_c.find('span', class_="listing-card__price__value").text.strip().replace('F Cfa', '')

        
        address = soup_c.find('span', class_="listing-item__address-location").text.strip()

        # Récupération du lien de l'image
        image_link = soup_c.find('img', class_="gallery__image__resource vh-img").get('src', '')

        dic = {
            'Etat': etat,
            'Marque': marque,
            'Annee': annee,
            'Boite_vitesse': transmission,
            'Prix': price,
            'Adresse': address,
            'Image_link': image_link
        }
        data.append(dic)

    except Exception as e:
        # print(f"Erreur lors du traitement d'un conteneur : {e}")
        pass

In [4]:
df = pd.DataFrame(data)
df.shape

(11, 7)

In [5]:
df = pd.DataFrame()

for page in range(1, 15):  
    url = url = f'https://www.expat-dakar.com/voitures?page={page}'
    res = get(url)
    soup = bs(res.text, 'html.parser')
    containers = soup.find_all('div', class_='listings-cards__list-item') 

    data = []
    for container in containers:
        try:
            url_container = container.find('a', class_='listing-card__inner')['href']
            res_c = get(url_container)
            soup_c = bs(res_c.text, 'html.parser')
         
            dd_elements = soup_c.find_all('dd', class_="listing-item__properties__description")
            
            marque = dd_elements[0].text.strip()
            modele = dd_elements[1].text.strip()
            etat = dd_elements[2].text.strip()
            transmission = dd_elements[3].text.strip()
            annee = dd_elements[4].text.strip()

            price = soup_c.find('span', class_="listing-card__price__value").text.strip().replace('F Cfa', '')
            address = soup_c.find('span', class_="listing-item__address-location").text.strip()
            image_link = soup_c.find('img', class_="gallery__image__resource vh-img").get('src', '')
           
            dic = {
                'Etat': etat,
                'Marque': marque,
                'Modele': modele,
                'Annee': annee,
                'Boite_vitesse': transmission,
                'Prix': price,
                'Adresse': address,
                'Image_link': image_link
            }
            data.append(dic)
        except Exception as e:
            pass
            # print(f"Erreur lors du traitement d'un conteneur : {e}")
    DF = pd.DataFrame(data)
    df = pd.concat([df,DF], axis=0).reset_index(drop=True)

Erreur lors du traitement d'un conteneur : 'NoneType' object has no attribute 'text'


In [6]:
df.to_csv('not_data_cleaning_vehicule.csv')

In [7]:
df

,Etat,Marque,Modele,Annee,Boite_vitesse,Prix,Adresse,Image_link
0,D'occasion,Nissan,Rogue,2016,Automatique,5 500 000,Ouest foire,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
1,Venant,Hyundai,Tucson,2024,Automatique,24 000 000,Almadies,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
2,Venant,Peugeot,208,2017,Automatique,5 000 000,Keur Massar,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
3,Venant,Citroen,C4,2016,Manuelle,5 800 000,Keur Massar,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
4,Venant,Mitsubishi,L200,2016,Manuelle,14 000 000,Keur Massar,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
...,...,...,...,...,...,...,...,...
148,D'occasion,Nissan,Rogue,2015,Automatique,4 900 000,Liberte 6,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
149,Venant,Jeep,Grand Cherokee,2015,Automatique,8 000 000,Rufisque,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
150,Automatique,Kia,Venant,"87,000 km",2016,8 450 000,Mermoz,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
151,Venant,Hyundai,Santa Fe,2017,Automatique,10 300 000,Mermoz,https://i.roamcdn.net/hz/ed/listing-thumb-543w...


In [10]:
df['Marque'].unique()

array(['Nissan', 'Hyundai', 'Peugeot', 'Citroen', 'Mitsubishi', 'BMW',
       'Mercedes-Benz', 'Ford', 'Mazda', 'Kia', 'Chevrolet', 'Land Rover',
       'Toyota', 'Renault', 'Jeep', 'Opel', 'Honda', 'Chery', 'Suzuki'],
      dtype=object)

In [6]:

def scrape_vehicules(pages):
    """Scrape les données des véhicules sur expat-dakar.com"""
    df = pd.DataFrame()
    for page in range(1, pages + 1):  
        url = f'https://www.expat-dakar.com/voitures?page={page}'
        res = get(url)
        soup = bs(res.text, 'html.parser')
        containers = soup.find_all('div', class_='listings-cards__list-item') 
        data = []
        for container in containers:
            try:
                url_container = container.find('a', class_='listing-card__inner')['href']
                res_c = get(url_container)
                soup_c = bs(res_c.text, 'html.parser')
                
                dd_elements = soup_c.find_all('dd', class_="listing-item__properties__description")
                
                marque = dd_elements[0].text.strip()
                modele = dd_elements[1].text.strip()
                etat = dd_elements[2].text.strip()
                transmission = dd_elements[3].text.strip()
                annee = dd_elements[4].text.strip()
                price = soup_c.find('span', class_="listing-card__price__value").text.strip().replace('F Cfa', '')
                address = soup_c.find('span', class_="listing-item__address-location").text.strip()
                image_link = soup_c.find('img', class_="gallery__image__resource vh-img").get('src', '')
                
                dic = {
                    'Etat': etat,
                    'Marque': marque,
                    'Modele': modele,
                    'Annee': annee,
                    'Boite_vitesse': transmission,
                    'Prix': price,
                    'Adresse': address,
                    'Image_link': image_link
                }
                data.append(dic)
            except Exception as e:
                # En cas d'erreur, on passe à l'élément suivant
                pass
        
        DF = pd.DataFrame(data)
        df = pd.concat([df, DF], axis=0).reset_index(drop=True)
    
    # Sauvegarde dans un fichier CSV
    df.to_csv("vehicules.csv", index=False)
    return df

In [7]:
d = scrape_vehicules(1)

In [8]:
d

,Etat,Marque,Modele,Annee,Boite_vitesse,Prix,Adresse,Image_link
0,Automatique,BMW,D'occasion,"128,000 km",2018,12 000 000,Ouakam,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
1,Automatique,Mercedes-Benz,Venant,"134,590 km",2020,34 500 000,Plateau,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
2,D'occasion,Hyundai,Tucson,2014,Automatique,5 700 000,VDN,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
3,Automatique,Autres,Venant,"150,000 km",2016,7 500 000,Parcelles Assainies,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
4,Venant,Hyundai,Elantra,2016,Automatique,4 800 000,VDN,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
5,D'occasion,Renault,Duster,2015,Automatique,4 490 000,Rufisque,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
6,Venant,Jeep,Grand Cherokee,2016,Automatique,12 000 000,Cité keur gorgui,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
7,Automatique,Ford,D'occasion,Essence,2017,6 500 000,Almadies,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
8,Automatique,Volkswagen,Venant,"100,000 km",2018,21 000 000,Dieuppeul,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
9,D'occasion,Hyundai,Elantra,2016,Automatique,4 900 000,Sicap Sacré-cœur,https://i.roamcdn.net/hz/ed/listing-thumb-543w...
